# Assignment 2 - DSE 310

## Submitted by - Mishika Lunia
## Roll no - 19194

In [1]:
import mysql.connector
from mysql.connector import Error

In [2]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error:
        print(f"Error: '{Error}'")

    return connection

In [3]:
password = "misha123#"
database_name = 'political_parties'
connection = create_server_connection("localhost", "misha123", password)

MySQL Database connection successful


In [4]:
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error:
        print(f"Error: '{Error}'")

In [5]:
create_database_query = f"CREATE DATABASE {database_name}"
create_database(connection, create_database_query)

Database created successfully


In [6]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error:
        print(f"Error: '{Error}'")

    return connection

In [7]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error:
        print(f"Error: '{Error}'")


In [8]:
create_parties_table = """
CREATE TABLE parties (
  party_id INT PRIMARY KEY,
  party_name VARCHAR(20) NOT NULL,
  chairman_name VARCHAR(20) NOT NULL,
  establishment_date DATE
  );
 """

connection = create_db_connection("localhost", "misha123", password, database_name)
execute_query(connection, create_parties_table)                       

MySQL Database connection successful
Query successful


In [9]:
create_leaders_table = """
CREATE TABLE leaders (
  party_id INT PRIMARY KEY,
  leader_name VARCHAR(40) NOT NULL,
  budget INT NOT NULL,
  FOREIGN KEY (party_id) REFERENCES parties(party_id)
  );
 """
connection = create_db_connection("localhost", "misha123", password, database_name)
execute_query(connection, create_leaders_table) 

MySQL Database connection successful
Query successful


### INSERT

In [23]:
data = """
INSERT INTO parties VALUES
(1,  'Aam Aadmi Party', 'Arvind Kejriwal', '2010-09-30'),
(2, 'BSP','Mayawati', '1926-07-09'),
(3, 'Biju Janata Dal', 'Naveen Pattnaik', '1990-11-12'),
(4, 'CPI',  'Sitaram Yechury', '1890-09-24')
"""

In [24]:
execute_query(connection, data)

Query successful


In [25]:
cursor = connection.cursor()
cursor.execute("SELECT * FROM parties;")
rows = cursor.fetchall()

In [26]:
for i in rows:
    print(i)

(1, 'Aam Aadmi Party', 'Arvind Kejriwal', datetime.date(2010, 9, 30))
(2, 'BSP', 'Mayawati', datetime.date(1926, 7, 9))
(3, 'Biju Janata Dal', 'Naveen Pattnaik', datetime.date(1990, 11, 12))
(4, 'CPI', 'Sitaram Yechury', datetime.date(1890, 9, 24))


In [32]:
cursor.execute("SELECT chairman_name FROM parties WHERE party_id = 1")
print(cursor.fetchall()[0][0])

Arvind Kejriwal


### UPDATE

In [37]:
cursor.execute("UPDATE parties SET chairman_name = 'Yogendra Yadav' WHERE party_id = 1;")

In [38]:
cursor.execute("SELECT * FROM parties;")
rows = cursor.fetchall()

In [39]:
for i in rows:
    print(i)

(1, 'Aam Aadmi Party', 'Yogendra Yadav', datetime.date(2010, 9, 30))
(2, 'BSP', 'Mayawati', datetime.date(1926, 7, 9))
(3, 'Biju Janata Dal', 'Naveen Pattnaik', datetime.date(1990, 11, 12))
(4, 'CPI', 'Sitaram Yechury', datetime.date(1890, 9, 24))


### Error when Foreign key Constraint is violated.

In [40]:
cursor.execute("INSERT INTO parties VALUES(5, 'CPI', '2025-03-16');")

DataError: 1136 (21S01): Column count doesn't match value count at row 1

### Dropping the Leaders Table

In [41]:
table_name = "leaders"

execute_query(connection, f"DROP TABLE {table_name}")

Query successful


### Adding User ('Mishika')

In [42]:
execute_query(connection, "CREATE USER 'mishika' IDENTIFIED BY 'mishi123#'")

Query successful


In [43]:
execute_query(connection, f"GRANT SELECT  ON {database_name}.parties TO mishika")

Query successful


In [44]:
connection2 = create_db_connection("localhost", "mishika", 'mishi123#', database_name) 

MySQL Database connection successful


In [45]:
cursor2 = connection2.cursor()
cursor2.execute("SHOW GRANTS;")
cursor2.fetchall()

[('GRANT USAGE ON *.* TO `mishika`@`%`',),
 ('GRANT SELECT ON `political_parties`.`parties` TO `mishika`@`%`',)]

### Removing privileges for user 'Mishika'

In [46]:
connection = create_db_connection("localhost", "misha123", password, database_name)

MySQL Database connection successful


In [48]:
cursor = connection.cursor()
cursor.execute(f"REVOKE ALL PRIVILEGES, GRANT OPTION FROM mishika")
cursor2.execute("SHOW GRANTS;")
cursor2.fetchall()

[('GRANT USAGE ON *.* TO `mishika`@`%`',)]

### Removing User 'Mishika'

In [49]:
cursor.execute("SELECT user FROM mysql.user;")
cursor.fetchall()

[('mishika',),
 ('misha123',),
 ('mysql.infoschema',),
 ('mysql.session',),
 ('mysql.sys',),
 ('root',)]

In [50]:
cursor.execute("DROP user mishika;")

In [51]:
cursor.execute("SELECT user FROM mysql.user;")
cursor.fetchall()

[('misha123',),
 ('mysql.infoschema',),
 ('mysql.session',),
 ('mysql.sys',),
 ('root',)]